# 1. Imports and setup

In [1]:
#Import libraries

import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.transform import Affine
from PIL import Image
import numpy as np
import os
import torch
import math
import torch.nn as nn

torch.random.manual_seed(0)

In [2]:
!nvidia-smi

Mon May  5 10:44:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8              11W / 120W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 2. Preprocessing of images

## 2.1. Cropping the splits

Geotiff images will be split in 512 by 512 tiles and PNGs

In [ ]:
def pad_array(array, target_height, target_width):
    bands, height, width = array.shape
    padded = np.zeros((bands, target_height, target_width), dtype=array.dtype)
    padded[:, :height, :width] = array
    return padded

def normalize(array):
    array_min, array_max = array.min(), array.max()
    if array_max - array_min == 0:
        return np.zeros_like(array, dtype=np.uint8)
    return ((array - array_min) / (array_max - array_min) * 255).astype(np.uint8)

def split_geotiff_with_padding(input_dir, filename, output_dir):
    tile_size=512
    os.makedirs(output_dir, exist_ok=True)
    png_dir = os.path.join(output_dir, "png")
    os.makedirs(png_dir, exist_ok=True)
    input_path = os.path.join(input_dir, filename)

    with rasterio.open(input_path) as src:
        width, height = src.width, src.height
        profile = src.profile

        for i in range(0, height, tile_size):
            for j in range(0, width, tile_size):
                window_height = min(tile_size, height - i)
                window_width = min(tile_size, width - j)
                window = Window(j, i, window_width, window_height)
                transform = src.window_transform(window)
                tile = src.read(window=window)

                if window_height < tile_size or window_width < tile_size:
                    tile = pad_array(tile, tile_size, tile_size)

                tile_profile = profile.copy()
                tile_profile.update({
                    'height': tile_size,
                    'width': tile_size,
                    'transform': transform
                })

                base_filename = os.path.splitext(filename)[0]
                tile_filename = f"{base_filename}_tile_{i // 512}_{j // 512}_p.tif"
                tile_path = os.path.join(output_dir, tile_filename)

                with rasterio.open(tile_path, 'w', **tile_profile) as dst:
                    dst.write(tile)

                # Generate PNG RGB
                if tile.shape[0] >= 3:
                    rgb = tile[:3]
                    rgb = normalize(rgb)
                    rgb = np.transpose(rgb, (1, 2, 0))  # Reorder to (H, W, Bands)
                    png_filename = f"{base_filename}_tile_{i//512}_{j//512}.png"
                    png_path = os.path.join(png_dir, png_filename)
                    Image.fromarray(rgb).save(png_path)


In [21]:
input_dir = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\input"
out_dir_allfiles = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\output"

In [ ]:
# Loop through all .tif files in input_dir
for file in os.listdir(input_dir):
    if file.endswith('.TIF') or file.endswith('.tif'):
        # Create a directory for each tiff file in out_dir_allfiles
        base_filename = os.path.splitext(file)[0]
        output_dir = os.path.join(out_dir_allfiles, f"{base_filename}_tiles")
        os.makedirs(output_dir, exist_ok=True)
        
        # Call the custom function to generate tiles and PNGs
        split_geotiff_with_padding(input_dir, file, output_dir)

## 2.2. Filtering the tiles

As some tiles may include heavily clouded sectors or dark sectors, they must be excluded from the pool

In [ ]:
# TODO: Filter the tiles